**Ge men er:**

I was trying to open these file by excel, that's silly and never work out LOL

All right, here is the problem. We need to predict** if a driver will file an insurance claim next year**. The train data file has 59 columns, 595212 rows; the test data file has 58 columns, and 892816 rows; and the sample_submission file has 2 columns and 892816 rows.

The only different column between train and test is 'target' column. The 2 columns in submission are id and target. These id in train and test has no overlap. 
 
 Ge Men er, **please** continue reading and feel free to **add comments**.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for drawing plot graph

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
#input the train set and test set
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

**Q1: What's the meaning of target?**


**MF Answer: ** I guess they want us to predict the probability of whether or not a claim was filed for the driver, so it should be in the range of [0,1]. 

Because I'm not sure if the target value could be 0.3, 0.8 or even 0.5. I made the plot gragh. The above graph shows the target value in train data file are either 1 or 0. Uh-Oh:(

But, in the sample_submission file, they have all the target value as 0.0364. Do you think it's a hint for us, the value could be any REAL value between [0,1]? 
Or we just rougly tend to think every driver's skill is as bad as YANG LAO SHI's. Thus, when target value less or equal than 0.5, we would give a piecewise to make it equals to 0 and elsewhere equals to 1.

I also find a smiliar discussion on https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/discussion/40222
, please search rkoptelov or Daniel Moller.

>>**HY Answer:** You can write anything here, such as agree:) Seriously, we can see the precentage of 0 and 1 are very different, probably 9:1. Do you think 0 means having a claim or not? Or it doesn't matter??? But how could that work?!

**Q2: What's the meaning of the other columns?**

We observed the column name is a combination of three or four parts, the fourth part(if has) could be cat or bin.

I find this answer from the** data description**. 

2.1  In the train and test data, features that belong to similar groupings are tagged as such in the feature names (e.g., ind, reg, car, calc). 

2.2 In addition, feature names include the postfix** bin to indicate binary features and cat to indicate categorical features**.

2.3  **Features without these designations are either continuous or ordinal**. Values of **-1** indicate that the feature was **missing** from the observation. 

>>Do you think we can do more in this part to understand features better, such as same first two parts, compute coefficient or we will compute each two anyhow. 

**Ge men er:**

Seriously,I agree with most of your points : ).Here is mine:

For train set, it is proper that the target values 0 or 1. The 0 means a driver will not initiate an auto insurance claim in the next year, And 1 means the opposite situation. For test set, we should give the probability that a driver will initiate an auto insurance claim in the next year.  We will order the user by probability and only the order counts. 

The models based on GBDT can predict the probability that the sample is classified to 1. 

If we use the xgboost or lightgbm, the coefficients between features can not influence our result. The missing data is fine for our model.

We don't know what the featrues represent. I think the only thing we can do is just dummies the category features, But I am worry about this rude method. We can do it by this method , check the result and think other method later.

The step of this question:

1) dummies the category features

2) check the features without the designations ,   continuous  or  category

3) put them into model Violently

Finally, try our best to fingure out the meanings of features.

To humor Mofei

In [ ]:
#for every category feature,check the number of different value and make sure the number is less than 10(10 is my
#own advice,we can change this value later).
train_col = list(train_df.columns)
for col in train_col:
    if col[-3:] == 'cat':
        values = train_df[col]
        if len(set(values)) > 10:
            print(col,len(set(values)))
# view the result, 'ps_car_11_cat' can't get dummies directly

>> **MF: I checked the result:**

>>ps_car_01_cat 13

>>ps_car_06_cat 18

>>ps_car_11_cat 104


>> I didn't see anything wrong. All the output column has more than 10  unique values.

>> But I still don't understand why more than 10 would be a problem. Could you explain more for me?


>> BTW, what are dummies meaning here?





In [ ]:
#check every features  category or continuous

ConVar = []
for col in train_col:
    values = train_df[col]
    if len(set(values)) > 30:
        ConVar.append(col)


In [ ]:
      for col in ConVar:
 print(col,len(set(train_df[col])))

>>**MF**: I understand above one step is checking the number of distinct value in all column, print out these columns witg numbers which are more than 30.
 Same question, what's the purpose of this step? I thought we will just jump to xgboost and try it without any processing.


In [ ]:
missvar = []
dis = len(train_df)
for col in train_col:
    values = train_df[col]
    miss = [-1 for x in values if x == -1]
    if len(miss)/dis > 0.5:
        print(len(miss))
        missvar.append(col)

>> Sorry, Heyang. I didn't get your point of above code. 

>> What's your purpose here?

>> If you want to print the count of all the missing then why do you make the condition >0,5?
>> Or you think only when the rate is more than 0.5, we need to deal with the missing value?

In [ ]:
values = train_df[missvar[0]]
values_ide = list(set(values))
print(values_ide)
print(missvar,len(set(values)),len(values))